In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.vision.all import *

In [ ]:
path = Path('/kaggle/input/appamomo2020-2024/AppaMomo')

datablock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(460),
    batch_tfms=aug_transforms(mult=1.0, do_flip=True, flip_vert=False,
                              max_rotate=30.0, min_zoom=1.0, max_zoom=1.1,
                              max_lighting=0.2, max_warp=0.2, p_affine=0.75,
                              p_lighting=0.75, xtra_tfms=None, size=224,
                              mode='bilinear', pad_mode='reflection', align_corners=True,
                              batch=False, min_scale=1.0)
)

dls = datablock.dataloaders(path, bs=32)

dls.show_batch(max_n=4)

In [ ]:
# Define the learner
learn = vision_learner(dls, resnet18, metrics=accuracy)

# Find an optimal learning rate
learn.lr_find()

In [ ]:
lr = 3e-3

learn.fine_tune(3, base_lr=lr)

In [ ]:
# Evaluate the model
learn.show_results()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(5)

In [ ]:
learn.export('new_model.pkl')

# Testing the model

In [ ]:
appa_test_image_path = '/kaggle/input/appamomo2020-2024/AppaMomo/Appa/IMG_6494.JPG'
momo_test_image_path = '/kaggle/input/appamomo2020-2024/AppaMomo/Momo/IMG_6639.JPG'

In [ ]:
# Load the exported model
model = load_learner('/kaggle/working/new_model.pkl')

In [ ]:
test_img = PILImage.create(momo_test_image_path)

# Predict using the model
pred, pred_idx, probs = model.predict(test_img)
print(f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}')